## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-18-21-51-10 +0000,nyt,‘No Kings’ Protests Against Trump: Photos and ...,https://www.nytimes.com/2025/10/18/us/politics...
1,2025-10-18-21-33-50 +0000,nyt,"Israeli Military Fires on Vehicle, Saying It C...",https://www.nytimes.com/2025/10/18/world/middl...
2,2025-10-18-21-23-51 +0000,nypost,Russia launches massive drone barrage overnigh...,https://nypost.com/2025/10/18/world-news/russi...
3,2025-10-18-21-19-52 +0000,bbc,Israel receives two bodies that Hamas says are...,https://www.bbc.com/news/articles/c7v8e1484n5o...
4,2025-10-18-21-15-40 +0000,nyt,"Manuel Trujillo, Who Helped New Yorkers Heal A...",https://www.nytimes.com/2025/10/18/nyregion/dr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,46
13,gaza,13
26,israel,12
35,new,11
136,ukraine,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
163,2025-10-18-03-30-12 +0000,nypost,Disgraced ex-Long Island Rep. George Santos re...,https://nypost.com/2025/10/17/us-news/george-s...,99
22,2025-10-18-19-41-44 +0000,startribune,Recap: Thousands join anti-Trump ‘No Kings’ pr...,https://www.startribune.com/live-thousands-exp...,93
218,2025-10-17-22-26-18 +0000,nypost,Trump orders lying ex-Long Island Rep. George ...,https://nypost.com/2025/10/17/us-news/trump-or...,92
11,2025-10-18-20-47-12 +0000,nyt,‘No Kings’ Protests Against Trump Draws Thousa...,https://www.nytimes.com/2025/10/18/us/protests...,90
184,2025-10-18-00-06-00 +0000,wsj,President Trump has commuted the sentence of f...,https://www.wsj.com/politics/trump-commutes-se...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
163,99,2025-10-18-03-30-12 +0000,nypost,Disgraced ex-Long Island Rep. George Santos re...,https://nypost.com/2025/10/17/us-news/george-s...
35,50,2025-10-18-18-55-54 +0000,latimes,Thousands gather for 'No Kings' protests acros...,https://www.latimes.com/california/story/2025-...
115,50,2025-10-18-10-45-00 +0000,wsj,Israel Still Carrying Out Some Strikes During ...,https://www.wsj.com/world/middle-east/israel-s...
199,43,2025-10-17-23-30-00 +0000,wsj,U.S. Is Holding Survivors of Latest Strike Aga...,https://www.wsj.com/world/americas/drug-boat-s...
116,39,2025-10-18-10-43-49 +0000,nyt,Ukraine Braces for New Talks Without the Lever...,https://www.nytimes.com/2025/10/18/world/europ...
29,32,2025-10-18-19-23-16 +0000,nypost,"Virginia Giuffre beaten, raped by ‘well-known ...",https://nypost.com/2025/10/18/us-news/virginia...
20,28,2025-10-18-19-51-55 +0000,nypost,Parents of Long Island teacher accused of sext...,https://nypost.com/2025/10/18/us-news/parents-...
34,28,2025-10-18-19-04-19 +0000,nypost,Obama-era Iran nuclear non-proliferation deal ...,https://nypost.com/2025/10/18/world-news/obama...
98,27,2025-10-18-12-21-13 +0000,nypost,Leaked texts from Oxford Union head who celebr...,https://nypost.com/2025/10/18/world-news/oxfor...
189,27,2025-10-17-23-50-52 +0000,nypost,Salesforce CEO Marc Benioff apologizes for say...,https://nypost.com/2025/10/17/us-news/salesfor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
